<a href="https://colab.research.google.com/github/boleamol/fake_news_classifier_using_LSTM/blob/main/Fake_News_Classifier_Using_Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake news classifier by using Bidirectional LSTM RNN**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')   # connecting to my google drive

Mounted at /content/gdrive


In [8]:
!pwd

/content


In [9]:
#changing the working directory and entering to dataset directory for my google drive
%cd drive/MyDrive/kaggle/fake_news/
!pwd

/content/drive/MyDrive/kaggle/fake_news
/content/drive/MyDrive/kaggle/fake_news


In [6]:
!ls

submit.csv  test.csv  train.csv


In [44]:
!pip install tensorflow===2.1.0

     |████████████████████████████████| 421.8 MB 22 kB/s 
     |████████████████████████████████| 448 kB 52.2 MB/s 
     |████████████████████████████████| 3.8 MB 39.5 MB/s 
     |████████████████████████████████| 50 kB 6.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=09cd817e341a937f3719ac7a7c58205fa0b380c37949f57151911470bfc04130
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.1.0'

In [10]:
import pandas as pd

In [11]:
df=pd.read_csv('train.csv')

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
###Drop Nan Values
df=df.dropna()

In [14]:
## Getting Independent Features in X
X=df.drop('label',axis=1)

In [15]:
## Get the Dependent features
y=df['label']

In [16]:
X.shape

(18285, 4)

In [17]:
y.shape

(18285,)

In [18]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [20]:
### Vocabulary size
voc_size=5000

In [21]:
messages=X.copy()

In [22]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [23]:
messages.reset_index(inplace=True)

In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [27]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3515, 1959, 2183, 4454, 3595, 2673, 3930, 399, 4758, 4447],
 [3756, 3271, 4060, 4331, 2472, 55, 1454],
 [2403, 2011, 4006, 1511],
 [4641, 3980, 4159, 4383, 2739, 4178],
 [4703, 2472, 3699, 2572, 3209, 1432, 2472, 4522, 631, 882],
 [530,
  2590,
  2137,
  2756,
  552,
  1349,
  2190,
  4360,
  3730,
  3889,
  4483,
  2534,
  1879,
  393,
  1454],
 [2393, 4674, 147, 1318, 3472, 1304, 4519, 3669, 4574, 2094, 1148],
 [2826, 3018, 1885, 4370, 2056, 1248, 1349, 2062, 4574, 2094, 1148],
 [3901, 4506, 2344, 4713, 638, 2944, 322, 2875, 1349, 2783],
 [1303, 707, 1685, 2641, 1481, 2123, 1899, 3865],
 [2195, 2367, 4143, 4359, 1798, 2875, 4293, 4261, 1991, 4885, 3147],
 [4383, 2903, 3595, 2944, 1349, 2056],
 [3026, 846, 1626, 882, 2075, 429, 3670, 836, 1798],
 [4217, 320, 31, 1544, 308, 4491, 2453, 4574, 2094, 1148],
 [908, 2306, 2456, 636, 4497, 4574, 2094, 1148],
 [1593, 3128, 1452, 1769, 548, 4040, 2991, 3803, 1409, 2453],
 [1404, 4668, 3271],
 [3649, 2770, 3787, 2127, 1349, 2533, 2769, 1454],

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  399 4758 4447]
 [   0    0    0 ... 2472   55 1454]
 [   0    0    0 ... 2011 4006 1511]
 ...
 [   0    0    0 ... 4574 2094 1148]
 [   0    0    0 ... 1976 1846 2032]
 [   0    0    0 ... 1043  754 3963]]


In [30]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3515,
       1959, 2183, 4454, 3595, 2673, 3930,  399, 4758, 4447], dtype=int32)

In [31]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [37]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/15
12250/12250 [==============================] - 18s 1ms/sample - loss: 0.3019 - accuracy: 0.8605 - val_loss: 0.2078 - val_accuracy: 0.9128
Epoch 2/15
12250/12250 [==============================] - 14s 1ms/sample - loss: 0.1403 - accuracy: 0.9451 - val_loss: 0.1969 - val_accuracy: 0.9167
Epoch 3/15
12250/12250 [==============================] - 14s 1ms/sample - loss: 0.0964 - accuracy: 0.9664 - val_loss: 0.2399 - val_accuracy: 0.9150
Epoch 4/15
12250/12250 [==============================] - 14s 1ms/sample - loss: 0.0616 - accuracy: 0.9786 - val_loss: 0.3493 - val_accuracy: 0.8973
Epoch 5/15
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.0382 - accuracy: 0.9870 - val_loss: 0.3270 - val_accuracy: 0.9117
Epoch 6/15
12250/12250 [==============================] - 15s 1ms/sample - loss: 0.0241 - accuracy: 0.9926 - val_loss: 0.4061 - val_accuracy: 0.9114
Epoch 7/15
12250/12250 [==============================] -

In [41]:
y_pred=model1.predict_classes(X_test)

In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test,y_pred)

array([[3088,  331],
       [ 212, 2404]])

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9100248550124275